In [ ]:
!pip install qiskit==0.19.6 qiskit-aer==0.5.2 pyzx --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install specific versions of Qiskit, Qiskit Aer, and PyZX
install("qiskit==0.19.6")
install("qiskit-aer==0.5.2")
install("pyzx")

import qiskit
from qiskit import QuantumCircuit, transpile, execute
from qiskit.providers.aer import Aer
import pyzx as zx

# Create a sample quantum circuit using Qiskit
def create_sample_circuit():
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.h(2)
    qc.measure_all()
    return qc

# Convert Qiskit circuit to PyZX graph
def qiskit_to_pyzx(qc):
    zx_circuit = zx.Circuit.from_qasm(qc.qasm())
    return zx_circuit

# Optimize the circuit using ZX-calculus
def optimize_circuit(zx_circuit):
    zx.simplify.full_reduce(zx_circuit)
    return zx_circuit

# Convert PyZX graph back to Qiskit circuit
def pyzx_to_qiskit(zx_circuit):
    optimized_qasm = zx_circuit.to_qasm()
    optimized_qc = QuantumCircuit.from_qasm_str(optimized_qasm)
    return optimized_qc

# Main function to demonstrate optimization
def main():
    # Create a sample quantum circuit
    qc = create_sample_circuit()
    print("Original Qiskit circuit:")
    print(qc)

    # Convert to PyZX graph
    zx_circuit = qiskit_to_pyzx(qc)

    # Optimize the circuit using ZX-calculus
    optimized_zx_circuit = optimize_circuit(zx_circuit)

    # Convert back to Qiskit circuit
    optimized_qc = pyzx_to_qiskit(optimized_zx_circuit)
    print("\nOptimized Qiskit circuit:")
    print(optimized_qc)

    # Execute the original and optimized circuits to compare results
    simulator = Aer.get_backend('qasm_simulator')
    original_result = execute(qc, simulator).result().get_counts()
    optimized_result = execute(optimized_qc, simulator).result().get_counts()

    print("\nOriginal circuit result:")
    print(original_result)
    print("\nOptimized circuit result:")
    print(optimized_result)

if __name__ == "__main__":
    main()


ImportError: cannot import name 'BaseProvider' from 'qiskit.providers' (/usr/local/lib/python3.10/dist-packages/qiskit/providers/__init__.py)

In [ ]:
!pip install pyzx --quiet


In [ ]:
import pyzx as zx

# Function to generate a random 5-qubit circuit
def generate_random_circuit(num_qubits):
    return zx.generate.cliffordT(qubits = num_qubits, depth = 50, p_t=None, p_s=None, p_hsh=None, p_cnot=None, backend=None)


# Function to optimize a circuit
def optimize_circuit(circuit):
    zx.simplify.full_reduce(circuit)
    return circuit

# Function to validate that the original and optimized circuits are the same
def validate_circuits(original, optimized):
    return zx.compare_tensors(original, optimized)

# Lists to store the original and optimized circuits
original_circuits = []
optimized_circuits = []

# Generate, optimize, and validate 10 circuits
for _ in range(10):
    original_circuit = generate_random_circuit(5)
    original_circuits.append(original_circuit.copy())  # Store the original circuit

    optimized_circuit = original_circuit.copy()  # Make a copy for optimization
    optimize_circuit(optimized_circuit)
    optimized_circuits.append(optimized_circuit)

    # Validate that the original and optimized circuits are the same
    if not validate_circuits(original_circuit, optimized_circuit):
        print("Validation failed for one of the circuits.")
    else:
        print("Circuit validated successfully.")

print("All circuits generated, optimized, and validated successfully.")


Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
Circuit validated successfully.
All circuits generated, optimized, and validated successfully.


In [ ]:
print(original_circuits)

[Graph(84 vertices, 93 edges), Graph(86 vertices, 97 edges), Graph(83 vertices, 91 edges), Graph(87 vertices, 99 edges), Graph(78 vertices, 81 edges), Graph(81 vertices, 87 edges), Graph(86 vertices, 97 edges), Graph(90 vertices, 105 edges), Graph(84 vertices, 93 edges), Graph(87 vertices, 99 edges)]


In [ ]:
print(optimized_circuits)

[Graph(23 vertices, 38 edges), Graph(22 vertices, 37 edges), Graph(22 vertices, 28 edges), Graph(22 vertices, 38 edges), Graph(19 vertices, 24 edges), Graph(25 vertices, 45 edges), Graph(24 vertices, 40 edges), Graph(21 vertices, 28 edges), Graph(26 vertices, 41 edges), Graph(26 vertices, 44 edges)]


In [ ]:
for j in original_circuits:
  zx.draw(j)

In [ ]:
for i in optimized_circuits:
  zx.draw(i)

In [ ]:
!pip install pyzx numpy pandas scikit-learn --quiet


In [ ]:
import pyzx as zx
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# Function to extract features from a circuit
def extract_features(circuit):
    num_gates = len(circuit.gates)
    num_qubits = circuit.qubits
    circuit_depth = circuit.depth()

    gate_counts = {}
    for gate in circuit.gates:
        gate_type = gate.name
        if gate_type in gate_counts:
            gate_counts[gate_type] += 1
        else:
            gate_counts[gate_type] = 1

    features = {
        'num_gates': num_gates,
        'num_qubits': num_qubits,
        'circuit_depth': circuit_depth,
    }

    # Add gate counts to features
    for gate_type, count in gate_counts.items():
        features[f'gate_count_{gate_type}'] = count

    return features

# Generate and optimize circuits
num_circuits = 10
original_circuits = []
optimized_circuits = []

for _ in range(num_circuits):
    original_circuit = zx.generate.cliffordT(qubits = 5, depth = 50, p_t=None, p_s=None, p_hsh=None, p_cnot=None, backend=None)
    original_circuits.append(original_circuit.copy())  # Store the original circuit

    optimized_circuit = original_circuit.copy()  # Make a copy for optimization
    zx.simplify.full_reduce(optimized_circuit)
    optimized_circuits.append(optimized_circuit)

# Extract features from the original and optimized circuits
original_features = [extract_features(circuit) for circuit in original_circuits]
optimized_features = [extract_features(circuit) for circuit in optimized_circuits]

# Convert features to a DataFrame
df_original = pd.DataFrame(original_features)
df_optimized = pd.DataFrame(optimized_features)

# Add labels to distinguish between original and optimized circuits
df_original['label'] = 0
df_optimized['label'] = 1

# Combine the datasets
df = pd.concat([df_original, df_optimized], ignore_index=True)

# Prepare data for training
X = df.drop(columns=['label'])
y = df['label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model using cross-validation
scores = cross_val_score(model, X, y, cv=5)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {scores.mean()}')

# Evaluate the model on the test set
test_score = model.score(X_test, y_test)
print(f'Test set score: {test_score}')


AttributeError: 'GraphS' object has no attribute 'gates'

In [ ]:
import pyzx as zx
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# Function to extract features from a circuit
def extract_features(circuit):
    num_vertices = circuit.num_vertices()
    num_edges = circuit.num_edges()
    num_inputs = len(circuit.inputs())
    num_outputs = len(circuit.outputs())

    # Gate counts can be derived from the vertex types
    vtypes = circuit.types()
    num_zx = sum(1 for t in vtypes if t == zx.VertexType.ZX)
    num_zx_z = sum(1 for t in vtypes if t == zx.VertexType.ZX_Z)
    num_zx_x = sum(1 for t in vtypes if t == zx.VertexType.ZX_X)

    features = {
        'num_vertices': num_vertices,
        'num_edges': num_edges,
        'num_inputs': num_inputs,
        'num_outputs': num_outputs,
        'num_zx': num_zx,
        'num_zx_z': num_zx_z,
        'num_zx_x': num_zx_x,
    }

    return features

# Generate and optimize circuits
num_circuits = 10
original_circuits = []
optimized_circuits = []

for _ in range(num_circuits):
    original_circuit = zx.generate.cliffordT(5, depth=10)
    original_circuits.append(original_circuit.copy())  # Store the original circuit

    optimized_circuit = original_circuit.copy()  # Make a copy for optimization
    zx.simplify.full_reduce(optimized_circuit)
    optimized_circuits.append(optimized_circuit)

# Extract features from the original and optimized circuits
original_features = [extract_features(circuit) for circuit in original_circuits]
optimized_features = [extract_features(circuit) for circuit in optimized_circuits]

# Convert features to a DataFrame
df_original = pd.DataFrame(original_features)
df_optimized = pd.DataFrame(optimized_features)

# Add labels to distinguish between original and optimized circuits
df_original['label'] = 0
df_optimized['label'] = 1

# Combine the datasets
df = pd.concat([df_original, df_optimized], ignore_index=True)

# Prepare data for training
X = df.drop(columns=['label'])
y = df['label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model using cross-validation
scores = cross_val_score(model, X, y, cv=5)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {scores.mean()}')

# Evaluate the model on the test set
test_score = model.score(X_test, y_test)
print(f'Test set score: {test_score}')


AttributeError: type object 'VertexType' has no attribute 'ZX'

In [ ]:
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=8, depth=100)
print(c.stats())
g = c.to_graph()
zx.full_reduce(g, quiet=False) # simplifies the Graph in-place, and show the rewrite steps taken.
g.normalize() # Makes the graph more suitable for displaying
zx.draw(g) # Display the resulting diagram

Circuit  on 8 qubits with 100 gates.
        19 is the T-count
        81 Cliffords among which
        66 2-qubit gates (66 CNOT, 0 other) and
        15 Hadamard gates.
spider_simp: 29. 18. 10. 11. 2. 3. 1.  7 iterations
id_simp: 13. 1.  2 iterations
spider_simp: 7. 4. 2.  3 iterations
pivot_simp: 12. 5. 3. 4. 2. 1. 1.  7 iterations
lcomp_simp: 1. 1.  2 iterations
